# 첫 번째 페이지 크롤링

In [1]:
import requests
from urllib import parse
from bs4 import BeautifulSoup

In [2]:
base_url = "https://movie.naver.com/movie/point/af/list.naver?&page={}"
url = base_url.format(1)

In [3]:
res = requests.get(url)

In [4]:
if res.status_code == 200:
    soup = BeautifulSoup(res.text)
    tds = soup.select('table.list_netizen > tbody > tr > td.title')
    print(len(tds))
    
    for td in tds:
        movie_title = td.select_one('a.movie').text.strip()
        link = td.select_one('a.movie').get('herf')
        link = parse.urljoin(base_url, link)
        score = td.select_one('div.list_netizen_score > em').text.strip()
        comment = td.select_one('br').next_sibling.strip()
        print(movie_title, link, score, comment, sep=' :: ')
        print('-------------------------------------------------------')

10
담보 :: https://movie.naver.com/movie/point/af/list.naver?&page={} :: 9 :: 너무 감동적이고 눈물콧물..딱 마지막 갈때쯔음 억지 스토리만 빼면 짱!
-------------------------------------------------------
듄 :: https://movie.naver.com/movie/point/af/list.naver?&page={} :: 10 :: 이 시리즈 영원히 안 끝났으면 좋겠다..SF영화 중 최고...
-------------------------------------------------------
레지스탕스 :: https://movie.naver.com/movie/point/af/list.naver?&page={} :: 10 :: 코로나 시기에 개봉이 안타깝다.나치나 일본 학살군이나 미치광이들의 지구 흔들기였다. 이제 더 이상 의미없는 희생은 없어야 한다. 각성하라,인간!
-------------------------------------------------------
듄 :: https://movie.naver.com/movie/point/af/list.naver?&page={} :: 8 :: 배경음악때문에몰입감이대박인영화
-------------------------------------------------------
클래식 :: https://movie.naver.com/movie/point/af/list.naver?&page={} :: 10 :: 처음부터 끝까지 감정선이 너무 좋았다.다시 개봉을 해도 손색이 없는 작품.조승우 손예진의 인생작이다.
-------------------------------------------------------
듄 :: https://movie.naver.com/movie/point/af/list.naver?&page={} :: 10 :: 진짜 이게 끝이라고??? 진짜 궁금했던거에 30퍼밖에 해소가 안됨 본편의 예고편

D:\program\anaconda3\lib\site-packages\bs4\__init__.py:177: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 194 of the file D:\program\anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  warnings.warn(self.NO_PARSER_SPECIFIED_WARNING % dict(


# 네이버 영화 평정과 리뷰 크롤링 후 저장

In [2]:
import random
import requests
import time
from bs4 import BeautifulSoup

In [6]:
random.uniform(0.2, 1.2)

0.7308643969712971

In [7]:
base_url = "https://movie.naver.com/movie/point/af/list.naver?&page={}"

In [8]:
# 결과 저장 list
comment_list = []

In [9]:
for page in range(1, 1001):
    url = base_url.format(page)
    res = requests.get(url)
    
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'lxml')
        tds = soup.select('table.list_netizen > tbody > tr > td.title')
        
        for td in tds:
            movie_title = td.select_one('a.movie').text.strip()
            link = td.select_one('a.movie').get('herf')
            link = parse.urljoin(base_url, link)
            score = td.select_one('div.list_netizen_score > em').text.strip()
            comment = td.select_one('br').next_sibling.strip()
            
            comment_list.append((movie_title, link, score, comment))
            
        # 크롤링 시 너무 자주 접근하면 사이트가 접근 방지할 수 있으므로 딜레이 걸어줌
        interval = round(random.uniform(0.2, 1.2), 2)
        time.sleep(interval)
        
print("finish")

finish


In [10]:
import pandas as pd

In [11]:
df = pd.DataFrame(comment_list, columns=['title', 'link', 'score', 'review'])
df.head()

,title,link,score,review
0,담보,https://movie.naver.com/movie/point/af/list.na...,9,너무 감동적이고 눈물콧물..딱 마지막 갈때쯔음 억지 스토리만 빼면 짱!
1,듄,https://movie.naver.com/movie/point/af/list.na...,10,이 시리즈 영원히 안 끝났으면 좋겠다..SF영화 중 최고...
2,레지스탕스,https://movie.naver.com/movie/point/af/list.na...,10,코로나 시기에 개봉이 안타깝다.나치나 일본 학살군이나 미치광이들의 지구 흔들기였다....
3,듄,https://movie.naver.com/movie/point/af/list.na...,8,배경음악때문에몰입감이대박인영화
4,클래식,https://movie.naver.com/movie/point/af/list.na...,10,처음부터 끝까지 감정선이 너무 좋았다.다시 개봉을 해도 손색이 없는 작품.조승우 손...


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   10000 non-null  object
 1   link    10000 non-null  object
 2   score   10000 non-null  object
 3   review  10000 non-null  object
dtypes: object(4)
memory usage: 312.6+ KB


In [13]:
df.to_csv('naver_movie_review_1102.csv', encoding='euckr', index=False)

# 특정 영화 리뷰 데이터 크롤링

In [26]:
base_url = "https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=113170&target=after&page={}"
url = base_url.format(1)

In [27]:
res = requests.get(url)

In [28]:
if res.status_code == 200:
    soup = BeautifulSoup(res.text)
    tds = soup.select('table.list_netizen > tbody > tr > td.title')
    print(len(tds))
    
    for td in tds:
        movie_title = td.select_one('a.movie').text.strip()
        link = td.select_one('a.movie').get('herf')
        link = parse.urljoin(base_url, link)
        score = td.select_one('div.list_netizen_score > em').text.strip()
        comment = td.select_one('br').next_sibling.strip()
        print(movie_title, link, score, comment, sep=' :: ')
        print('-------------------------------------------------------')

10
마담 프루스트의 비밀정원 :: https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=113170&target=after&page={} :: 9 :: 추천 받아서 봤는데 생각보다 매우 좋았어요. 추천합니다.
-------------------------------------------------------
마담 프루스트의 비밀정원 :: https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=113170&target=after&page={} :: 10 :: 내 인생 3대영화아바타, 프로메테우스 그리고 마담프루스
-------------------------------------------------------
마담 프루스트의 비밀정원 :: https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=113170&target=after&page={} :: 10 :: 나중에 자식을 낳았을때 아이가 15살이 되면 보여주고 싶은 영화
-------------------------------------------------------
마담 프루스트의 비밀정원 :: https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=113170&target=after&page={} :: 10 :: 진짜 좋은영화 잔잔하면서도 주옥같은 ost
-------------------------------------------------------
마담 프루스트의 비밀정원 :: https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=113170&target=after&page={} :: 7 :: 음악 소리와 힐링하는 과정이 아름다운 잔잔한 영화
-----------------------------

In [17]:
random.uniform(0.2, 1.2)

0.9894644113514122

In [18]:
base_url = "https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword=113170&target=after&page={}"

In [19]:
# 결과 저장 list
comment_list = []

In [20]:
for page in range(1, 250):
    url = base_url.format(page)
    res = requests.get(url)
    
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'lxml')
        tds = soup.select('table.list_netizen > tbody > tr > td.title')
        
        for td in tds:
            movie_title = td.select_one('a.movie').text.strip()
            link = td.select_one('a.movie').get('herf')
            link = parse.urljoin(base_url, link)
            score = td.select_one('div.list_netizen_score > em').text.strip()
            comment = td.select_one('br').next_sibling.strip()
            
            comment_list.append((movie_title, link, score, comment))
            
        # 크롤링 시 너무 자주 접근하면 사이트가 접근 방지할 수 있으므로 딜레이 걸어줌
        interval = round(random.uniform(0.2, 1.2), 2)
        time.sleep(interval)
        
print("finish")

finish


In [10]:
import pandas as pd

In [21]:
df = pd.DataFrame(comment_list, columns=['title', 'link', 'score', 'review'])
df.head()

,title,link,score,review
0,마담 프루스트의 비밀정원,https://movie.naver.com/movie/point/af/list.na...,9,추천 받아서 봤는데 생각보다 매우 좋았어요. 추천합니다.
1,마담 프루스트의 비밀정원,https://movie.naver.com/movie/point/af/list.na...,10,"내 인생 3대영화아바타, 프로메테우스 그리고 마담프루스"
2,마담 프루스트의 비밀정원,https://movie.naver.com/movie/point/af/list.na...,10,나중에 자식을 낳았을때 아이가 15살이 되면 보여주고 싶은 영화
3,마담 프루스트의 비밀정원,https://movie.naver.com/movie/point/af/list.na...,10,진짜 좋은영화 잔잔하면서도 주옥같은 ost
4,마담 프루스트의 비밀정원,https://movie.naver.com/movie/point/af/list.na...,7,음악 소리와 힐링하는 과정이 아름다운 잔잔한 영화


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2490 entries, 0 to 2489
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   2490 non-null   object
 1   link    2490 non-null   object
 2   score   2490 non-null   object
 3   review  2490 non-null   object
dtypes: object(4)
memory usage: 77.9+ KB


In [29]:
df.to_csv('naver_movie_review_1104_AttilaMarcel.csv', encoding='euckr', index=False)